In [ ]:
# dependencies
import re
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
import matplotlib.pyplot as plt

from zipfile import ZipFile
from matplotlib.pyplot import figure
from collections import Counter, defaultdict
from itertools import combinations

from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity as cosine
from sklearn.metrics.pairwise import euclidean_distances as euclidean

In [ ]:
# installing Character BERT
!git clone https://github.com/helboukkouri/character-bert.git

In [ ]:
# installing Transformers for CharacterBert
! pip install transformers

## Loading the Data

In [ ]:
# loading the dat
userData = pd.read_csv("anonymized_disqus_user_data.csv")
botComments = pd.read_csv("botnet_comments.csv")
# potentialBot = pd.read_csv("potential_social_botnets.csv")
# metaData = pd.read_csv("user_meta_data.csv")

In [ ]:
# converting message to string as some of them are float, etc.
botComments.message = botComments.message.astype(str)

In [ ]:
# removing the html tags
botComments.message = botComments.message.apply(lambda html: re.sub('<.*?>', '', html))

## Bot Detection and Metrics

### Clustering UserNames

In [ ]:
%cd ./character-bert/

In [ ]:
!pwd

In [ ]:
from modeling.character_bert import CharacterBertModel
from utils.character_cnn import CharacterIndexer

In [ ]:
!python download.py --model='general_character_bert'

In [ ]:
# Load some pre-trained CharacterBERT
model = CharacterBertModel.from_pretrained(
    './pretrained-models/general_character_bert/')

In [ ]:
# np.array of all usernames
x = botComments.username.drop_duplicates().to_numpy()
n_batch = min(x.shape[0]//512, 60)
batch = x[:n_batch*512].reshape((n_batch, 512))


# Convert token sequence into character indices
indexer = CharacterIndexer()
batch_ids = indexer.as_padded_tensor(batch)

# Feed batch to CharacterBERT & get the embeddings
embeddings_for_batch, _ = model(batch_ids)
embeddings_for_x = embeddings_for_batch

In [ ]:
userNameEmbeddings = embeddings_for_x.reshape(-1, embeddings_for_x.shape[-1]).detach().numpy()

In [ ]:
userNameEmbeddings.shape

In [ ]:
# Agglomerative Clustering (Heirarchial Clustering)
clusteringModel = AgglomerativeClustering(n_clusters=None, distance_threshold=11.9)
clusteringModel.fit(userNameEmbeddings)
clusterAssignment = clusteringModel.labels_

In [ ]:
# processing clustered data
clusteredUsers = defaultdict(set)
for userId, clusterId in enumerate(clusterAssignment):
    clusteredUsers[clusterId].add(x[userId])

In [ ]:
len(clusteredUsers)

In [ ]:
clusteredUsers

In [ ]:
euclidean([userNameEmbeddings[991]], [userNameEmbeddings[992]])

In [ ]:
cosine([userNameEmbeddings[991]], [userNameEmbeddings[992]])

### Metrics

In [ ]:
# cluster based on same password
def clusterByPassword(botDF, userData):
  bots = botDF[['username']].drop_duplicates()
  botsWithDetails = pd.merge(botDF, userData, on='username')
  groupedBots = botsWithDetails.groupby(by='password_id').agg(set)
  expectedCluster = {}
  for id, cluster in enumerate(groupedBots.username.values):
    expectedCluster[id] = cluster
  return expectedCluster

In [ ]:
# defining function to get true/predicted edges
def getEdges(clusters):
  allEdges = set()
  for cluster in clusters.values():
    edges = set(combinations(cluster, 2))
    allEdges |= edges
  return allEdges

In [ ]:
# metric definition
def getMetric(n, trueEdges, predictedEdges):
  totalEdges = n*((n - 1)/2)
  trueEdges = set(map(tuple, map(sorted, trueEdges)))
  predictedEdges = set(map(tuple, map(sorted, predictedEdges)))
  truePositives = trueEdges & predictedEdges
  falsePositives = predictedEdges - trueEdges
  print("TE/PE:", len(trueEdges), len(predictedEdges))
  print("TP/FP:", len(truePositives), len(falsePositives))
  accuracy = len(truePositives)/len(trueEdges)
  penalizingFactor = 1 - ((len(falsePositives)*len(truePositives))/totalEdges)
  return accuracy*penalizingFactor

In [ ]:
expectedCluster = clusterByPassword(botComments, userData)

In [ ]:
trueEdges = getEdges(expectedCluster)

In [ ]:
predictedEdges = getEdges(clusteredUsers)

In [ ]:
n = pd.merge(botComments[['username']].drop_duplicates(), userData, on='username').shape[0]

In [ ]:
getMetric(n, trueEdges, predictedEdges)

In [ ]:
userNameToEmbedding = {k: v for k, v in zip(x, userNameEmbeddings)}

In [ ]:
def getEmbedding(username):
    return userNameToEmbedding[username]

In [ ]:
embeddings = botComments.username.apply(lambda username: getEmbedding(username))

In [ ]:
embeddings = embeddings.to_numpy()

In [ ]:
embeddings = np.stack(embeddings)

In [ ]:
embeddings.shape

In [ ]:
x.shape

In [ ]:
rem_batch = [x[29184:]]

batch_ids_ = indexer.as_padded_tensor(rem_batch)

# Feed batch to CharacterBERT & get the embeddings
embeddings_for_batch_, _ = model(batch_ids_)
embeddings_for_x_ = embeddings_for_batch[0]

In [ ]:
embeddings_for_x_ = embeddings_for_x_.detach().numpy()

In [ ]:
for k, v in zip(rem_batch, embeddings_for_x_):
    userNameToEmbedding[k] = v

In [ ]:
rem_batch = rem_batch[0].tolist()

In [ ]:
botComments.shape

In [ ]:
%cd ..

In [ ]:
np.save("userEmbeddings.npy", embeddings)